In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.models.DCVC_net import DCVC_net
import torch
from torchvision import transforms
import numpy as np
import pathlib
import os
import matplotlib.pyplot as plt
import tqdm

In [6]:
LAMBDA = 2048
BATCH_SIZE = 1
DATA_DIR = pathlib.Path('/data2/jatin/vimeo_septuplet/sequences')
DEVICE = torch.device('cpu')
DEVICE

device(type='cpu')

In [9]:
video_net = DCVC_net()

In [10]:
# load the good weights
# video_net.opticFlow = torch.load('../DCVC-Old/optflow.pth')

video_net = video_net.to(DEVICE)

optimizer = torch.optim.AdamW(video_net.parameters(), lr=1e-4)


In [11]:
mse_criterion = torch.nn.MSELoss()

In [17]:
x = torch.rand((2, 3, 256, 256))
x = x.unsqueeze(dim=0)
x = x.to(DEVICE)

In [18]:
ref = x[:,0]
im = x[:,1]

In [21]:
# TEST MEMC
preds_dict = video_net(ref, im, compress_type='train_compress', train_type='memc')
mse = mse_criterion(preds_dict['pred'], im)
mse.backward()
optimizer.step()

In [23]:
# TEST MEMC + MV BPP
preds_dict = video_net(ref, im, compress_type='train_compress', train_type='memc_bpp')
mse = mse_criterion(preds_dict['pred'], im)
loss = mse * LAMBDA + preds_dict['mv_z_bpp'] + preds_dict['mv_y_bpp']
loss.backward()
optimizer.step()

In [24]:
# TEST RECON
preds_dict = video_net(ref, im, compress_type='train_compress', train_type='recon')
mse = mse_criterion(preds_dict['recon_image'], im)
mse.backward()
optimizer.step()

In [27]:
# TEST FULL
preds_dict = video_net(ref, im, compress_type='train_compress', train_type='full')
mse = mse_criterion(preds_dict['recon_image'], im)
loss = mse * LAMBDA + preds_dict['bpp']
loss.backward()
optimizer.step()